In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "Qwen/Qwen2.5-3B-Instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name,force_download=True, use_default_system_prompt=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 38.50it/s]


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

c:\Program Files\Python311-64\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pipe = pipeline(
    "text-generation",
    "Qwen/Qwen2.5-3B-Instruct",
    temperature=0.6,
    max_new_tokens=32000,
    device="cuda:0",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 46.05it/s]
Device set to use cuda:0


In [11]:
from src.config.prompts import main_prompt
from src.tools.registry import ToolRegistry

In [12]:
prompt = main_prompt.format(tools_description = ToolRegistry().get_tools_description(), history = '', input = "")

<class 'langchain_core.tools.structured.StructuredTool'>
<class 'langchain_core.tools.structured.StructuredTool'>
<class 'langchain_core.tools.structured.StructuredTool'>


In [13]:
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": 'how 1000*123/333333'}
]

In [14]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)

NameError: name 'tokenizer' is not defined

In [123]:
print(text)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
how 1000*123/333333<|im_end|>
<|im_start|>assistant



In [96]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [97]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

In [98]:
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

In [99]:
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

In [100]:
thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

In [101]:
print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: To solve the expression \(1000 \times 123 / 333333\), you can follow these steps:

1. First, multiply 1000 by 123:
   \[
   1000 \times 123 = 123000
   \]

2. Next, divide the result by 333333:
   \[
   123000 / 333333 \approx 0.36894736842105263
   \]

So, \(1000 \times 123 / 333333\) is approximately \(0.36894736842105263\).


In [17]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

In [18]:
m = ChatHuggingFace(llm=HuggingFacePipeline(pipeline=pipe))

In [ ]:
prompt_template = PromptTemplate(
    template='{input}',
    input_variables=['input'],
    

)

In [21]:
s = SystemMessage(content="четкий поц")

In [ ]:
(prompt_template | m).invoke({"input": "Привет!"})

AIMessage(content='<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nПривет!<|im_end|>\n<|im_start|>assistant\nЗдравствуйте! Как я могу вам помочь сегодня?', additional_kwargs={}, response_metadata={}, id='run--b85a8844-be69-410c-ae24-784faec7265b-0')

In [ ]:
prompt

ChatPromptTemplate(input_variables=['"tool"', "'title'"], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['"tool"', "'title'"], input_types={}, partial_variables={}, template='You are an intelligent assistant with access to the following tools:\n\nHISTORY:\n\n\nYOUR AVAILABLE TOOLS:\nget_weather:\n   - Description: Get the current weather for the specified city.\n        Args:\n            input_data: The name of the city in English (for example: \'Moscow\', \'Samara\')\n\n        Returns:\n            A line with information about temperature and weather conditions\n        \n   - Input data: input: {\'title\': \'Input\'}\n\ncalculate:\n   - Description: Calculate a mathematical expression.\n        Args:\n            input_data: A mathematical expression (for example: \'2 + 2 * 3\', \'sin(45)\')\n\n        Returns:\n            Calculation result\n        \n   - Input data: input: {\'title\': \'Input\'}\n\nsearch_info

In [25]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

m = ChatHuggingFace(llm=HuggingFacePipeline(pipeline=pipe))

# Создаем чат-промпт с системным сообщением
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="Ты - экспертный AI ассистент по программированию. Ты помогаешь с кодом на Python."),
    HumanMessage(content="{input}")
])

# Создаем цепочку
chain = chat_prompt | m

# Используем
response = chain.invoke({"input": "Как написать функцию на Python?"})

In [26]:
response

AIMessage(content='<|im_start|>system\nТы - экспертный AI ассистент по программированию. Ты помогаешь с кодом на Python.<|im_end|>\n<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\nКажется, вы забыли указать что именно вам нужна помощь. Вы можете вставить свой код или вопрос о том, как решить определенную задачу. Если у вас есть конкретный код или проблема, я буду рад помочь!', additional_kwargs={}, response_metadata={}, id='run--e1eaa6b0-f7f5-495e-90f2-c2276423a2e8-0')